In [2]:
from plotly.subplots import make_subplots
from plotting import plot_values, update_layout
import glob
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
import json
import os
import matplotlib.pyplot as plt

ENVS = [
    "SafetyHopperVelocity-v1",
    "SafetyHalfCheetahVelocity-v1",
    "SafetyHumanoidVelocity-v1",
    "SafetySwimmerVelocity-v1",
    "SafetyAntVelocity-v1",
    "SafetyWalker2dVelocity-v1",
]

In [3]:
out = []
for env in ENVS:
    frames = []
    for idx, file in enumerate(glob.glob(f"../data/{env}/*/log.txt")):
        frames.append(pd.read_csv(file, sep='\t', usecols=["Iteration", "AverageReward", "gradnorms", "maxnorms"]))
        frames[-1]["idx"] = idx
    df = pd.concat(frames, ignore_index=True).groupby("idx", as_index=False).agg({
        "AverageReward": ["max"],
        "maxnorms": ["max"],
        "gradnorms": ["max"]
    })
    df.columns = ["_".join([x,y]) for x, y in df.columns]
    df = df[["idx_", "AverageReward_max", "maxnorms_max", "gradnorms_max"]]
    df["env"] = env.replace("Safety", "").replace("Velocity-v1", "")
    out.append(df)
df = pd.concat(out)
df = df.groupby("env", as_index=False)[["maxnorms_max", "gradnorms_max"]].agg(["min", "median"])
df

maxnorms_max            gradnorms_max           
                     min     median           min     median
env                                                         
Ant             0.702969   1.857426      2.591170   4.855397
HalfCheetah     0.740927   9.575896      1.549630  10.217608
Hopper          1.323620   9.945652      1.717533   3.915468
Humanoid        0.943106   5.101860     13.738074  45.642847
Swimmer         1.078192  34.830896      1.466625  23.369379
Walker2d        0.708152   2.082297      1.762007   4.229802

In [6]:
out = []
for env in ENVS:
    frames = []
    for idx, file in enumerate(glob.glob(f"../data/{env}/*/log.txt")):
        frames.append(pd.read_csv(file, sep='\t', usecols=["Time"]))
        frames[-1]["idx"] = idx
    df = pd.concat(frames, ignore_index=True).groupby("idx", as_index=False).Time.max()
    df["env"] = env.replace("Safety", "").replace("Velocity-v1", "")
    out.append(df)
df = pd.concat(out)
df.groupby("env").Time.describe()

,count,mean,std,min,25%,50%,75%,max
env,,,,,,,,
Ant,8.0,12204.778511,695.435090,11254.891920,11793.095641,12167.142194,12418.162280,13548.318098
HalfCheetah,16.0,7580.919586,893.659039,6156.233756,6948.471105,7820.694780,8173.216451,8788.451824
Hopper,16.0,5178.948364,2308.387790,3526.008355,3710.771038,4413.367235,4935.365585,11308.728322
Humanoid,7.0,34565.045753,14839.362627,9900.398968,27013.251247,34435.698437,45746.123797,52100.472775
Swimmer,36.0,3723.325793,945.085926,2424.266868,3094.267110,3681.000856,4007.572242,5911.216912
Walker2d,24.0,7791.133390,2023.015472,5202.923935,6083.336890,7194.007962,9842.667946,11761.070541
